In [12]:
import pandas as pd
import numpy as np
df = pd.read_json('purchase_data.json')
df.head(10)
#len(df.columns) # 6total columns 

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


In [2]:
#player count: total number of non unique screen names  ---number of unique screen names 780
df.SN.count()

780

In [9]:
#number of unique items 179
df["Item ID"].nunique()

183

In [11]:
#average purchase price is $2.93
df.Price.mean()

2.931192307692303

In [5]:
#total number of purchases is equal to total number of screennames  
#as eachscreenname represents one purchase. 
df["SN"].count()

780

In [6]:
#total revenue is $2286.33 
df.Price.sum()

2286.33

In [7]:
#total number of males is 633
#total number of females is 136
#total number of nondisclosed is 11
grouped =df.groupby("Gender").count().reset_index()
grouped.head()

,Gender,Age,Item ID,Item Name,Price,SN
0,Female,136,136,136,136,136
1,Male,633,633,633,633,633
2,Other / Non-Disclosed,11,11,11,11,11


In [8]:
#percentage demographics
#males 81.15%
#females 17.43% 
#nondisclosed 1.41%
df["Gender"].value_counts(normalize=True) * 100

Male                     81.153846
Female                   17.435897
Other / Non-Disclosed     1.410256
Name: Gender, dtype: float64

In [21]:
#Purchasing analysis by gender-----MALES
male_purch = df[df["Gender"] == "Male"]["Price"].count()
male_avgpurch = df[df["Gender"] == "Male"]['Price'].mean()
male_purchtotal = df[df["Gender"] == "Male"]['Price'].sum()
totalmales = df[df["Gender"] == "Male"]["SN"].nunique()
male_normalized = male_purchtotal/totalmales
male_purch, male_avgpurch, male_purchtotal, male_normalized

#male purchase count is 633, with an average purchase price of $2.95
#total male purchase value is $1867.68 with a normalized total of $4.02

(633, 2.9505213270142154, 1867.68, 4.016516129032258)

In [36]:
#Purchasing analysis-----FEMALES
female_purch = df[df["Gender"] == "Female"]["Price"].count()
female_avgpurch = df[df["Gender"] == "Female"]['Price'].mean()
female_purchtotal = df[df["Gender"] == "Female"]['Price'].sum()
totalfemales = df[df["Gender"] == "Female"]["SN"].nunique()
female_normalized = female_purchtotal/totalfemales
female_purch, female_avgpurch,female_purchtotal,female_normalized

#female purchase count is 136 with an everage purchase price of $2.81 
#total male purchase value is $382.91 with a normalized total of $3.83

(136, 2.815514705882352, 382.90999999999997, 3.8290999999999995)

In [49]:
#other/non disclosed purchase analysis
totalpurchases = df['Price'].count()
NDpurch = totalpurchases - male_purch - female_purch
ND_avgpurch = df[df["Gender"] == "Other / Non-Disclosed"]['Price'].mean()
ND_purchtotal = df[df["Gender"] == "Other / Non-Disclosed"]['Price'].sum()
totalpeople = df["SN"].nunique()
othercount = totalpeople - totalmales - totalfemales
other_normalized = ND_purchtotal/othercount

NDpurch, ND_avgpurch,ND_purchtotal,other_normalized
#Non disclosed purchase count is 11 with an everage purchase price of $3.25
#total nondisclosed purchase value is $35.74 with a normalized total of $4.47

(11, 3.2490909090909086, 35.739999999999995, 4.467499999999999)

In [45]:
#AGE DEMOGRAPHICS--------BROKEN DOWN BY 4 YEAR BINS 
bins = [0, 9, 14, 19, 24,29,34,39,100]
group_names = ['<10', '10-14', '15-19', '20-24','25-29','30-34','35-39','40+']
pd.cut(df["Age"], bins, labels=group_names)
df["Age Group"] = pd.cut(df["Age"], bins, labels=group_names)

pymoli_groupby_sn_agegrp = df.groupby(["SN","Age Group"])

                                                              
pymoli_unique_payers_df = pd.DataFrame(pymoli_groupby_sn_agegrp.size())
#print(pymoli_unique_payers_df)
pymoli_agegrp_df = pd.DataFrame(pymoli_unique_payers_df.groupby(["Age Group"]).count())
pymoli_agegrp_df.columns= ["Total Count"]
pymoli_agegrp_df["Percentage of Players"] =round(100 *pymoli_agegrp_df["Total Count"]/pymoli_agegrp_df["Total Count"].sum(),2)

pymoli_agegrp_df


,Total Count,Percentage of Players
Age Group,,
<10,19,3.32
10-14,23,4.01
15-19,100,17.45
20-24,259,45.20
25-29,87,15.18
30-34,47,8.20
35-39,27,4.71
40+,11,1.92


In [47]:
#PURCHASING ANALYSIS 
pymoli_agegrp_not_sn_df = pd.DataFrame(df.groupby(["Age Group"]).count())
pymoli_agegrp_sum_not_sn_df = pd.DataFrame(df.groupby(["Age Group"]).sum())
pymoli_agegrp_df["Purchase Count"] =  pymoli_agegrp_not_sn_df["Item ID"]
pymoli_agegrp_df["Average Purchase Price"] = round(pymoli_agegrp_sum_not_sn_df["Price"]/pymoli_agegrp_df["Purchase Count"],2).map("${:,.2f}".format)
pymoli_agegrp_df["Total Purchase Value"] = pymoli_agegrp_sum_not_sn_df["Price"]
pymoli_agegrp_df["Normalized Totals"] =  round(pymoli_agegrp_df["Total Purchase Value"]/pymoli_agegrp_df["Total Count"],2).map("${:,.2f}".format)
pymoli_agegrp_df["Total Purchase Value"] = pymoli_agegrp_df["Total Purchase Value"].map("${:,.2f}".format)
pymoli_agegrp_df = pymoli_agegrp_df.drop(labels = ["Total Count","Percentage of Players"],axis = 1)
pymoli_agegrp_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


In [51]:
#TOP SPENDERS IN THE GAME
pymoli_counts_df = pd.DataFrame(df.groupby("SN").count())
pymoli_sum_df = pd.DataFrame(df.groupby("SN").sum())
pymoli_spenders_df = pymoli_sum_df
pymoli_spenders_df = pymoli_spenders_df.sort_values(by=["Price"], ascending=False)
pymoli_spenders_df["Purchase Count"] = pymoli_counts_df["Item ID"]
pymoli_spenders_df["Average Purchase Price"] = round(pymoli_spenders_df["Price"]/pymoli_spenders_df["Purchase Count"],2).map("${:,.2f}".format)
pymoli_spenders_df["Total Purchase Value"] = pymoli_spenders_df["Price"].map("${:,.2f}".format)
pymoli_spenders_df = pymoli_spenders_df.drop(labels = ["Age","Item ID","Price"],axis = 1)
pymoli_spenders_df.head(10)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58
Isursti83,3,$3.68,$11.05
Isurria36,3,$3.67,$11.01
Eusri70,3,$3.52,$10.55
Aerithllora36,3,$3.48,$10.45


In [52]:
#MOST POPULAR ITEMS IN THE GAME 
pymoli_items_count_df = pd.DataFrame(df.groupby(["Item ID","Item Name"]).count())
pymoli_items_sum_df = pd.DataFrame(df.groupby(["Item ID","Item Name"]).sum())
pymoli_items_count_df["Purchase Count"] = pymoli_items_count_df["Age"]
pymoli_items_count_df["Item Price"] = round(pymoli_items_sum_df["Price"]/pymoli_items_count_df["Age"],2)
pymoli_items_count_df["Total Purchase Value"] = pymoli_items_sum_df["Price"]
pymoli_items_count_df = pymoli_items_count_df.sort_values(by = ["Purchase Count","Total Purchase Value"],ascending = False)
pymoli_popular_items_df = pymoli_items_count_df.drop(labels = ["Age","Age Group","Gender","Price","SN"],axis = 1)
pymoli_popular_items_df["Item Price"] = pymoli_popular_items_df["Item Price"].map("${:,.2f}".format)
pymoli_popular_items_df["Total Purchase Value"] = pymoli_popular_items_df["Total Purchase Value"].map("${:,.2f}".format)
pymoli_popular_items_df.head()

,,Age_Group,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,,
39,"Betrayal, Whisper of Grieving Widows",11,11,$2.35,$25.85
84,Arcane Gem,11,11,$2.23,$24.53
34,Retribution Axe,9,9,$4.14,$37.26
31,Trickster,9,9,$2.07,$18.63
13,Serenity,9,9,$1.49,$13.41


In [53]:
#MOST PROFITABLE ITEMS IN THE GAME 
pymoli_items_count_df = pymoli_items_count_df.sort_values(by = ["Total Purchase Value"],ascending = False)
pymoli_profitable_items_df = pymoli_items_count_df.drop(labels = ["Age","Age Group","Gender","Price","SN"],axis = 1)
pymoli_profitable_items_df["Item Price"] = pymoli_profitable_items_df["Item Price"].map("${:,.2f}".format)
pymoli_profitable_items_df["Total Purchase Value"] = pymoli_profitable_items_df["Total Purchase Value"].map("${:,.2f}".format)
pymoli_profitable_items_df.head()

,,Age_Group,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,,
34,Retribution Axe,9,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,7,$4.25,$29.75
32,Orenmir,6,6,$4.95,$29.70
103,Singed Scalpel,6,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,8,$3.61,$28.88
